In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from random import seed
from random import randrange
import seaborn as sns

In [19]:
# Lectura de datos
data = sns.load_dataset('iris')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [20]:
# Cambiar ajuste de datos para clasificar setosa
data['categoria'] = [1 if x =='setosa' else -1 for x in data['species']] 

In [21]:
# Función clasifica
def clasifica(X, Y, theta, theta_0):
    
    """ Funcion que realiza la evaluacion de la prediccion contra las eitquetas:
    theta - parametro theta
    theta_0 - parametro theta0
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)    

    Outputs:
    value = -1 en caso de tener una clasificacion erronea. 0 eoc
    """
    
    if Y*(np.dot(theta,np.transpose(X)) + theta_0) < 1 :
        value = -1
    else:
        value = 0

    return value

In [32]:
def loess(X, Y, theta, theta_0):
    """ Funcion que realiza la evaluacion en en la funcion de perdida:
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)        
    theta - parametro theta
    theta_0 - parametro theta0

    Outputs:
    value= valor de perdida
     """
    
    suma = 0
    H = 0
    n = X.shape[0]
    
    for i in range(n):
        H = Y[i]*(np.dot(theta, X[i]) + theta_0)
        if H >= 1:
            suma += 0
        else:
            suma += (1-H)
            
    return suma

In [33]:
def jacob(X, Y, theta, theta_0, lamb):
    """ Funcion que realiza la evaluacionen de la clasiificaciòn lineal con margen como optimizacion
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)        
    theta- parametro theta
    theta0 - parametro theta0
    lamb - parametro de regularizacion

    Outputs:
    value = valor del jacobiano
     """
    
    n = X.shape[0]
    jac = loess(X, Y, theta, theta_0)/n + (lamb/2.0)*np.linalg.norm(theta)

    return jac

In [51]:
# Función para descenso por gradiente
def SVM(X, Y, theta, theta_0, etha, lamb, eps = 1e-8, MAX = 5000):
    """ Funcion que realiza el aglrotimo de Support Vector Machine - Descenso en Gradiente
    Inputs:
    X - vector de características a clasificar
    Y - vector de etiquetas de las características (-1 o 1)    
    theta - incializacion de parametro theta
    theta_0 - inicializacion de parametro theta0
    etha - tasa de aprendizaje
    lamb - parametro de regularizacion
    
    Outputs:
    tetha = vector para formar el plano
    tetha_0 = escalar para desplazar el plano 
    """
    
    n = X.shape[0]
    sum_theta = 0
    sum_theta_0 = 0
    error = 10
    t = 0
    
    while error >= eps and t < MAX:
        # Guardar parametros en tiempo: t-1
        theta_old = theta
        theta_0_old = theta_0
        
        # Evaluacion de la suma para la actualizacion el descenso por gradiente
        for i in range(n):
            sum_theta += clasifica(X[i], Y[i], theta, theta_0)*Y[i]*X[i]
            sum_theta_0 += clasifica(X[i], Y[i], theta, theta_0)*Y[i]
        
        # Descenso por gradiente
        theta = theta - etha*((sum_theta/n) + lamb*theta)
        theta_0= theta_0 - etha*((sum_theta_0/n))
        
        # Calculo del error
        error = abs(jacob(X, Y, theta, theta_0, lamb) - jacob(X, Y, theta_old, theta_0_old, lamb))
        t += 1
        
    return theta, theta_0, t, error

In [ ]:
def SGD(X, Y, theta, lamb, eps = 1e-8, T = 5000):
    """ Funcion que los parámetros del modelo perceptron 
    Inputs:
    X_array - arreglo de caracteristicas numericas
    y_array - arreglo etiquetas de las características (-1 o 1)
    Nota: se asume que X y y tienen la misma cantidad de registros
    
    T -  Número de iteraciones
    lambda_ - Párametro lambda 
    
    Outputs:
    theta_t - parámetro theta e intercepto (theta_t[-1])
    
    """
    
    n = X.shape[0]
    #ones = np.matrix(np.ones((X.shape[0], 1)))
    #X = np.append(X, ones, axis=1)
    #Y = np.matrix(Y).reshape(Y.shape[0],1)
    t = 0
    error = 10
    
    while error >= eps and t <= T:
        # Guardar parametros en tiempo: t-1
        theta_old = theta
        
        i = np.random.randint(n)
        etha = 1/(t+1)
        theta = theta - etha*((np.dot(theta, X[i]) - Y[i])*X[i] + (lamb/n)*theta)
        t += 1
        
        # Calculo del error
        error = abs(jacob(X, Y, theta, theta_0, lamb) - jacob(X, Y, theta_old, theta_0_old, lamb))
        
    return np.array(theta_t)

In [76]:
# Separar datos para prueba y entrenamiento
train = data.sample(frac = 0.8, random_state = 2020) # Fijamos la semilla con random_state
test = data.drop(train.index)

# Extraer variables para la clasificación
data_vars = train.iloc[:,0:4]
vars_train = data_vars.to_numpy()
data_labels = train.iloc[:,5]
labels_train = data_labels.to_numpy()

theta = np.array([0.0, 0.0, 0.0, 0.0])
theta_0 = 0.0
etha = 0.0001
lamb = 0.0001